---
layout: post  
---

In this post, I will demonstrate two methods for how edges could be stored in a graph genome
- explicitly storing edges
- inferring possible edges during evaluation

and argue for explicitly storing observed edges rather than inferring possible edges during runtime

To begin, I'll simulate a (very) small genome, determine the kmers (nodes of the graph) present in the genome, and link them

In [1]:
import Pkg
pkgs = [
    "DataStructures",
    "BioSequences",
    "GraphRecipes",
    "LightGraphs",
    "PlotlyJS",
    "Plots",
    "Random",
    "StatsBase",
    "StatsPlots"
]

Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

Plots.plotlyjs()

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %9 %           ]  70.1 %

  Resolving package versions...


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-8821867106595839692\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-2588700107597847346\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-8821867106595839692\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
┌ Warning: ORCA.jl has been deprecated and all savefig functionality
│ has been implemented directly in PlotlyBase itself.
│ 
│ By implementing in PlotlyBase.jl, the savefig routines are automatically
│ available to PlotlyJS.jl also.
└ @ ORCA /home/jupyter-cjprybol/.julia/packages/ORCA/U5XaN/src/ORCA.jl:8


Plots.PlotlyJSBackend()

In [2]:
Random.seed!(0)

Random.MersenneTwister(UInt32[0x00000000], Random.DSFMT.DSFMT_state(Int32[748398797, 1073523691, -1738140313, 1073664641, -1492392947, 1073490074, -1625281839, 1073254801, 1875112882, 1073717145  …  943540191, 1073626624, 1091647724, 1073372234, -1273625233, -823628301, 835224507, 991807863, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000

In [3]:
genome = BioSequences.randdnaseq(10)

10nt DNA Sequence:
TGTATACTGG

In [4]:
k = 3

3

In [5]:
kmer_counts = StatsBase.countmap(BioSequences.canonical(kmer.fw) for kmer in BioSequences.each(BioSequences.DNAMer{k}, genome))

Dict{Any,Int64} with 6 entries:
  GTA => 2
  ATA => 2
  ACA => 1
  ACT => 1
  CAG => 1
  CCA => 1

In [6]:
kmers = DataStructures.OrderedDict(
    kmer => (index = index, count = count) for (index, (kmer, count)) in enumerate(sort(kmer_counts))
)

OrderedCollections.OrderedDict{BioSequences.Mer{BioSequences.DNAAlphabet{2},3},NamedTuple{(:index, :count),Tuple{Int64,Int64}}} with 6 entries:
  ACA => (index = 1, count = 1)
  ACT => (index = 2, count = 1)
  ATA => (index = 3, count = 2)
  CAG => (index = 4, count = 1)
  CCA => (index = 5, count = 1)
  GTA => (index = 6, count = 2)

In [7]:
K = length(kmers)

6

In [8]:
graph = LightGraphs.SimpleGraph(K)
for i in 1:length(genome)-k
    a_to_b_connection = genome[i:i+k]
    a = BioSequences.canonical(BioSequences.DNAMer(a_to_b_connection[1:end-1]))
    b = BioSequences.canonical(BioSequences.DNAMer(a_to_b_connection[2:end]))
    a_index = kmers[a].index
    b_index = kmers[b].index
    edge = LightGraphs.Edge(a_index, b_index)
    LightGraphs.add_edge!(graph, edge)
end
# Note hover doesn't really work well for these because it cycles the hover info over the edges as well
GraphRecipes.graphplot(
    graph,
    names = 1:K,
    markersize = 0.15,
    fontsize=12)

<!DOCTYPE html>
 
 
 Plots.jl

In this basic example, we can see that if we iterate through all kmer connections in our original dataset to determine the set of edges, we are able to resolve a single path that will reconstruct the original sequence.

```
1 <-> 6 <-> 3 <-> 3 <-> 6 <-> 2 <-> 4 <-> 5
```

While that sequence can be read in the forward or reverse-complement orientation (i.e. we can reverse the order in which we visit each node), that path visits each node at least once and in the fewest # of steps.
The length of the path will also generate a sequence equal to the length of our original genome.

The algorithmic runtime of assessing the edges of a given graph using the above framework is proportional to the size of the dataset:

$$\text{Runtime} \propto \text{# of Reads}$$

Determining the edges requires evaluating each pairwise kmer cominbation in the dataset, which we've already done (at least) once in order to count the kmers.

Some genome assemblers don't store the edges at all. Instead, the algorithms infer whether an edge could exist based on whether the two kmers are neighbors. We define "neighbors" as two kmers that can satisfiy the condition
```
kmer_a[2:end] == kmer_b[1:end-1]
```

which is that the` k-1` length _suffix_ of `kmer_a` is equal to the `k-1` length _prefix_ of `kmer_b` where `kmer_a` and `kmer_b` can be in either forward or reverse_complement orientations

Below, we'll show the same dataset represented with a graph that explicitly stores all of the possible kmer connections that would be inferred as potentially existing in some frameworks

In [9]:
graph = LightGraphs.SimpleGraph(K)
# This may be wrong too. We shouldn't necessarily add edges just becuase they COULD exist
# Maybe we want to only add edges when they DO exist
for (index_a, fw_kmer) in enumerate(kmers.keys)
    for kmer in (fw_kmer, BioSequences.reverse_complement(fw_kmer))
        for neighbor in BioSequences.neighbors(kmer)
            canonical_neighbor = BioSequences.canonical(neighbor)
            index_b = get(kmers, canonical_neighbor, (index = 0, count = 0)).index
            if index_b != 0
                LightGraphs.add_edge!(graph, LightGraphs.Edge(index_a, index_b))
            end
        end
    end
end
GraphRecipes.graphplot(
    graph,
    names = 1:K,
    markersize = 0.15,
    fontsize=12)

<!DOCTYPE html>
 
 
 Plots.jl

As seen above in this simplified example, if we store all possible edges, we have the potential to add additional structural complexity to the graph that isn't supported by the data. As the size of the kmer increases, the likelihood that any two kmers could exist in a dataset _without_ being linked decreases asymptotically towards 0. With small kmer sizes and large datasets, however, the risk of falsely linking kmers could be very high.

Since we've already made the concession that we will only store and consider kmers that were actually observed to exist, rather than the list of kmers that _could_ exist, I would argue that we should only entertain edges that were also demonstrated to exist.